<a href="https://colab.research.google.com/github/candle16/statcan-crime-analysis/blob/main/Canada_Crime_Rate_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Canada Crime Rate Analysis (2000-2024)

This notebook analyzes crime rate trends in Canada using Statistics Canada data (Table 35-10-0177-01).

**Outputs:**
1. Line graph showing crime rate trends (2000-2024) for total crime, top violent crime, and top property crime
2. Table ranking Top 20 violent Criminal Code violations in 2024 with growth metrics
3. Table ranking Top 20 property violations in 2024 with growth metrics

**Data Source:** [Statistics Canada Table 35-10-0177-01](https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=3510017701)

In [ ]:
# Install required packages
!pip install -q pandas matplotlib requests openpyxl

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import zipfile
from io import BytesIO
from pathlib import Path
from IPython.display import display, HTML

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully")

## 1. Fetch Crime Rate Data from Statistics Canada

In [ ]:
# Statistics Canada Table 35-10-0177-01: Incident-based crime statistics
TABLE_ID = "35100177"

# Construct download URL
download_url = f"https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid={TABLE_ID}01&latestN=0&startDate=&endDate=&csvLocale=en&selectedMembers=%5B%5B%5D%2C%5B%5D%2C%5B%5D%5D"

print(f"Downloading data from Statistics Canada (Table {TABLE_ID})...")

try:
    response = requests.get(download_url, timeout=60)
    response.raise_for_status()
    
    # Extract ZIP file
    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
        csv_files = [f for f in zip_file.namelist() if f.endswith('.csv')]
        
        if not csv_files:
            raise ValueError("No CSV file found in downloaded ZIP")
        
        csv_filename = csv_files[0]
        print(f"Extracted: {csv_filename}")
        
        with zip_file.open(csv_filename) as csv_file:
            df = pd.read_csv(csv_file)
    
    print(f"Data loaded: {len(df):,} rows, {len(df.columns)} columns")
    print(f"Time period: {df['REF_DATE'].min()} to {df['REF_DATE'].max()}")
    
except Exception as e:
    print(f"Error: {e}")
    raise

In [ ]:
# Explore data structure
print("Columns:", df.columns.tolist())
print("\nSample data:")
df.head()

In [ ]:
# Check unique values in key columns
print("Unique GEO values:")
print(df['GEO'].unique())
print(f"\nUnique Statistics: {df['Statistics'].nunique()}")
print(df['Statistics'].unique())

## 2. Filter and Prepare Data

In [ ]:
# Filter for Canada-wide data and crime rate statistics
df_canada = df[
    (df['GEO'] == 'Canada') & 
    (df['Statistics'] == 'Rate per 100,000 population')
].copy()

# Convert year
df_canada['Year'] = df_canada['REF_DATE'].astype(int)

# Filter for 2000-2024
df_canada = df_canada[df_canada['Year'].between(2000, 2024)]

# Remove rows with missing values
df_canada = df_canada[df_canada['VALUE'].notna()]

print(f"Filtered data: {len(df_canada):,} rows")
print(f"Years: {df_canada['Year'].min()} to {df_canada['Year'].max()}")
print(f"Unique violations: {df_canada['Violations'].nunique()}")

In [ ]:
# Explore violation categories
print("Sample violation types:")
violations = df_canada['Violations'].unique()
for v in violations[:30]:
    print(f"  - {v}")

In [ ]:
# Define categories based on Statistics Canada classification
# Violent crimes are typically under "Violent Criminal Code violations"
# Property crimes are under "Property crime"

# Find key aggregates
total_crime_patterns = ['Total Criminal Code violations (excluding traffic)', 'Total, all Criminal Code violations']
violent_category = 'Total violent Criminal Code violations'
property_category = 'Total property crime'

# Check what exists
print("Looking for key categories...")
for pattern in total_crime_patterns + [violent_category, property_category]:
    matches = df_canada[df_canada['Violations'].str.contains(pattern, case=False, na=False, regex=False)]
    if len(matches) > 0:
        print(f"Found: '{pattern}' - {len(matches)} rows")

In [ ]:
# List all violations to identify violent and property crimes
print("All unique violations:")
for i, v in enumerate(sorted(violations), 1):
    print(f"{i:3}. {v}")

## 3. Identify Crime Categories

In [ ]:
# Define violent and property crime lists based on Criminal Code classification
# These are common Criminal Code violations categorized by type

violent_crimes = [
    'Homicide', 'Murder, first degree', 'Murder, second degree', 'Manslaughter',
    'Attempted murder', 'Conspiracy to commit murder',
    'Sexual assault, level 3, aggravated', 'Sexual assault, level 2, weapon or bodily harm',
    'Sexual assault, level 1', 'Sexual violations against children',
    'Assault, level 3, aggravated', 'Assault, level 2, weapon or bodily harm',
    'Assault, level 1', 'Assault peace officer',
    'Other assaults', 'Firearms - use of, discharge, pointing',
    'Robbery', 'Forcible confinement or kidnapping', 'Abduction',
    'Extortion', 'Criminal harassment', 'Uttering threats',
    'Indecent or harassing communications', 'Non-consensual distribution of intimate images'
]

property_crimes = [
    'Breaking and entering', 'Theft over $5,000 (non-motor vehicle)',
    'Theft of $5,000 or under (non-motor vehicle)', 'Theft of motor vehicle',
    'Possession of stolen property', 'Fraud', 'Identity fraud',
    'Mischief', 'Arson'
]

# Find actual matches in the data
def find_matching_violations(pattern_list, all_violations):
    """Find violations that contain any of the patterns."""
    matches = []
    for v in all_violations:
        for pattern in pattern_list:
            if pattern.lower() in v.lower():
                matches.append(v)
                break
    return matches

matched_violent = find_matching_violations(violent_crimes, violations)
matched_property = find_matching_violations(property_crimes, violations)

print(f"Matched violent crimes: {len(matched_violent)}")
print(f"Matched property crimes: {len(matched_property)}")

In [ ]:
# Get data for different crime categories
# Total crime
df_total = df_canada[df_canada['Violations'].str.contains('Total Criminal Code violations', case=False, na=False)]
if len(df_total) == 0:
    df_total = df_canada[df_canada['Violations'].str.contains('Total, all violations', case=False, na=False)]

# Violent crime total
df_violent_total = df_canada[df_canada['Violations'].str.contains('Total violent', case=False, na=False)]

# Property crime total  
df_property_total = df_canada[df_canada['Violations'].str.contains('Total property', case=False, na=False)]

print(f"Total crime rows: {len(df_total)}")
print(f"Violent crime total rows: {len(df_violent_total)}")
print(f"Property crime total rows: {len(df_property_total)}")

if len(df_total) > 0:
    print(f"\nTotal crime category: {df_total['Violations'].iloc[0]}")
if len(df_violent_total) > 0:
    print(f"Violent total category: {df_violent_total['Violations'].iloc[0]}")
if len(df_property_total) > 0:
    print(f"Property total category: {df_property_total['Violations'].iloc[0]}")

In [ ]:
# Find top individual violent and property crimes in 2024
df_2024 = df_canada[df_canada['Year'] == 2024].copy()

# Filter for specific violent crimes (not totals)
df_violent_2024 = df_2024[
    df_2024['Violations'].isin(matched_violent) & 
    ~df_2024['Violations'].str.contains('Total', case=False, na=False)
].copy()

# Filter for specific property crimes (not totals)
df_property_2024 = df_2024[
    df_2024['Violations'].isin(matched_property) & 
    ~df_2024['Violations'].str.contains('Total', case=False, na=False)
].copy()

# Get top violent crime by rate
if len(df_violent_2024) > 0:
    top_violent = df_violent_2024.nlargest(1, 'VALUE')['Violations'].iloc[0]
    print(f"Top violent crime in 2024: {top_violent}")
else:
    top_violent = None
    print("No violent crimes found")

# Get top property crime by rate
if len(df_property_2024) > 0:
    top_property = df_property_2024.nlargest(1, 'VALUE')['Violations'].iloc[0]
    print(f"Top property crime in 2024: {top_property}")
else:
    top_property = None
    print("No property crimes found")

## 4. Create Line Graph: Crime Rate Trends (2000-2024)

In [ ]:
# Prepare data for line graph
# Get total crime trend
total_trend = df_total.groupby('Year')['VALUE'].first().reset_index()
total_trend.columns = ['Year', 'Total Crime Rate']

# Get top violent crime trend
if top_violent:
    violent_trend = df_canada[df_canada['Violations'] == top_violent].groupby('Year')['VALUE'].first().reset_index()
    violent_trend.columns = ['Year', 'Top Violent Crime Rate']
else:
    violent_trend = pd.DataFrame({'Year': [], 'Top Violent Crime Rate': []})

# Get top property crime trend
if top_property:
    property_trend = df_canada[df_canada['Violations'] == top_property].groupby('Year')['VALUE'].first().reset_index()
    property_trend.columns = ['Year', 'Top Property Crime Rate']
else:
    property_trend = pd.DataFrame({'Year': [], 'Top Property Crime Rate': []})

print("Data prepared for visualization")
print(f"Total crime years: {len(total_trend)}")
print(f"Violent crime years: {len(violent_trend)}")
print(f"Property crime years: {len(property_trend)}")

In [ ]:
# Create the line graph
fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot total crime rate on primary y-axis
color_total = '#1f77b4'
line1 = ax1.plot(total_trend['Year'], total_trend['Total Crime Rate'], 
                 marker='o', linewidth=2.5, markersize=6, 
                 label='Total Criminal Code Violations', color=color_total)

ax1.set_xlabel('Year', fontsize=12, fontweight='bold')
ax1.set_ylabel('Crime Rate (per 100,000 population)', fontsize=12, fontweight='bold')

# Create secondary y-axis for individual crimes (different scale)
ax2 = ax1.twinx()

# Plot top violent crime
color_violent = '#d62728'
if len(violent_trend) > 0:
    line2 = ax2.plot(violent_trend['Year'], violent_trend['Top Violent Crime Rate'], 
                     marker='s', linewidth=2.5, markersize=6, linestyle='--',
                     label=f'{top_violent}', color=color_violent)

# Plot top property crime
color_property = '#2ca02c'
if len(property_trend) > 0:
    line3 = ax2.plot(property_trend['Year'], property_trend['Top Property Crime Rate'], 
                     marker='^', linewidth=2.5, markersize=6, linestyle='-.',
                     label=f'{top_property}', color=color_property)

ax2.set_ylabel('Individual Crime Rate (per 100,000 population)', fontsize=12, fontweight='bold')

# Title
plt.title('Crime Rate Trends in Canada (2000-2024)\nTotal Crime vs Top Violent and Property Crimes', 
          fontsize=14, fontweight='bold', pad=20)

# Format x-axis
years = list(range(2000, 2025, 2))
ax1.set_xticks(years)
ax1.set_xticklabels(years, rotation=45)

# Grid
ax1.grid(True, alpha=0.3, linestyle='--')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=10, framealpha=0.9)

# Format y-axes with commas
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

plt.tight_layout()
plt.savefig('crime_rate_trends_2000_2024.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nLine graph saved as 'crime_rate_trends_2000_2024.png'")

## 5. Top 20 Violent Criminal Code Violations (2024)

In [ ]:
# Get all violent crime data for 2024 and 2019
# Broader search for violent crimes
violent_keywords = [
    'homicide', 'murder', 'manslaughter', 'assault', 'sexual', 'robbery',
    'kidnapping', 'confinement', 'abduction', 'extortion', 'harassment',
    'threats', 'firearm', 'weapon', 'violence', 'violent'
]

def is_violent_crime(violation):
    """Check if violation is a violent crime."""
    v_lower = violation.lower()
    # Exclude totals and property crimes
    if 'total' in v_lower or 'property' in v_lower:
        return False
    return any(keyword in v_lower for keyword in violent_keywords)

# Get 2024 violent crimes
df_violent_2024 = df_2024[df_2024['Violations'].apply(is_violent_crime)].copy()

# Get 2019 data
df_2019 = df_canada[df_canada['Year'] == 2019].copy()
df_violent_2019 = df_2019[df_2019['Violations'].apply(is_violent_crime)].copy()

print(f"Violent crimes in 2024: {len(df_violent_2024)}")
print(f"Violent crimes in 2019: {len(df_violent_2019)}")

In [ ]:
# Create Top 20 Violent Crimes table
# Get top 20 by 2024 crime rate
top_20_violent = df_violent_2024.nlargest(20, 'VALUE')[['Violations', 'VALUE']].copy()
top_20_violent.columns = ['Violation', 'Crime Rate 2024']

# Add 2019 rates and calculate growth
growth_data = []
for _, row in top_20_violent.iterrows():
    violation = row['Violation']
    rate_2024 = row['Crime Rate 2024']
    
    # Get 2019 rate
    rate_2019_data = df_violent_2019[df_violent_2019['Violations'] == violation]['VALUE']
    rate_2019 = rate_2019_data.iloc[0] if len(rate_2019_data) > 0 else 0
    
    # Calculate growth
    if rate_2019 > 0:
        growth = ((rate_2024 - rate_2019) / rate_2019) * 100
    else:
        growth = np.nan
    
    growth_data.append({
        'Rank': len(growth_data) + 1,
        'Violent Crime Violation': violation,
        'Crime Rate 2024': rate_2024,
        'Crime Rate 2019': rate_2019,
        'Growth 2019-2024 (%)': growth
    })

df_violent_table = pd.DataFrame(growth_data)
print("Top 20 Violent Criminal Code Violations table created")

In [ ]:
# Display formatted table
print("=" * 100)
print("TOP 20 VIOLENT CRIMINAL CODE VIOLATIONS IN CANADA (2024)")
print("Ranked by Crime Rate (per 100,000 population)")
print("=" * 100)

# Format for display
df_violent_display = df_violent_table.copy()
df_violent_display['Crime Rate 2024'] = df_violent_display['Crime Rate 2024'].apply(lambda x: f'{x:,.1f}')
df_violent_display['Crime Rate 2019'] = df_violent_display['Crime Rate 2019'].apply(lambda x: f'{x:,.1f}')
df_violent_display['Growth 2019-2024 (%)'] = df_violent_display['Growth 2019-2024 (%)'].apply(
    lambda x: f'{x:+.1f}%' if pd.notna(x) else 'N/A'
)

# Style the table
styled_violent = df_violent_display.style.set_properties(**{
    'text-align': 'left',
    'font-size': '11px'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#d62728'), ('color', 'white'), 
                                  ('font-weight', 'bold'), ('text-align', 'center'), ('padding', '8px')]},
    {'selector': 'td', 'props': [('padding', '6px'), ('border', '1px solid #ddd')]},
    {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#f9f9f9')]}
]).hide(axis='index')

display(styled_violent)

# Save to CSV
df_violent_table.to_csv('top_20_violent_crimes_2024.csv', index=False)
print("\nTable saved as 'top_20_violent_crimes_2024.csv'")

## 6. Top 20 Property Violations (2024)

In [ ]:
# Get all property crime data
property_keywords = [
    'theft', 'break', 'entering', 'robbery', 'fraud', 'mischief', 'arson',
    'stolen property', 'motor vehicle', 'shoplifting', 'property'
]

def is_property_crime(violation):
    """Check if violation is a property crime."""
    v_lower = violation.lower()
    # Exclude totals
    if 'total' in v_lower:
        return False
    # Exclude violent crimes that might overlap
    if any(kw in v_lower for kw in ['assault', 'sexual', 'murder', 'homicide']):
        return False
    return any(keyword in v_lower for keyword in property_keywords)

# Get 2024 property crimes
df_property_2024 = df_2024[df_2024['Violations'].apply(is_property_crime)].copy()

# Get 2019 property crimes
df_property_2019 = df_2019[df_2019['Violations'].apply(is_property_crime)].copy()

print(f"Property crimes in 2024: {len(df_property_2024)}")
print(f"Property crimes in 2019: {len(df_property_2019)}")

In [ ]:
# Create Top 20 Property Crimes table
top_20_property = df_property_2024.nlargest(20, 'VALUE')[['Violations', 'VALUE']].copy()
top_20_property.columns = ['Violation', 'Crime Rate 2024']

# Add 2019 rates and calculate growth
property_growth_data = []
for _, row in top_20_property.iterrows():
    violation = row['Violation']
    rate_2024 = row['Crime Rate 2024']
    
    # Get 2019 rate
    rate_2019_data = df_property_2019[df_property_2019['Violations'] == violation]['VALUE']
    rate_2019 = rate_2019_data.iloc[0] if len(rate_2019_data) > 0 else 0
    
    # Calculate growth
    if rate_2019 > 0:
        growth = ((rate_2024 - rate_2019) / rate_2019) * 100
    else:
        growth = np.nan
    
    property_growth_data.append({
        'Rank': len(property_growth_data) + 1,
        'Property Crime Violation': violation,
        'Crime Rate 2024': rate_2024,
        'Crime Rate 2019': rate_2019,
        'Growth 2019-2024 (%)': growth
    })

df_property_table = pd.DataFrame(property_growth_data)
print("Top 20 Property Violations table created")

In [ ]:
# Display formatted table
print("=" * 100)
print("TOP 20 PROPERTY VIOLATIONS IN CANADA (2024)")
print("Ranked by Crime Rate (per 100,000 population)")
print("=" * 100)

# Format for display
df_property_display = df_property_table.copy()
df_property_display['Crime Rate 2024'] = df_property_display['Crime Rate 2024'].apply(lambda x: f'{x:,.1f}')
df_property_display['Crime Rate 2019'] = df_property_display['Crime Rate 2019'].apply(lambda x: f'{x:,.1f}')
df_property_display['Growth 2019-2024 (%)'] = df_property_display['Growth 2019-2024 (%)'].apply(
    lambda x: f'{x:+.1f}%' if pd.notna(x) else 'N/A'
)

# Style the table
styled_property = df_property_display.style.set_properties(**{
    'text-align': 'left',
    'font-size': '11px'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2ca02c'), ('color', 'white'), 
                                  ('font-weight', 'bold'), ('text-align', 'center'), ('padding', '8px')]},
    {'selector': 'td', 'props': [('padding', '6px'), ('border', '1px solid #ddd')]},
    {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#f9f9f9')]}
]).hide(axis='index')

display(styled_property)

# Save to CSV
df_property_table.to_csv('top_20_property_crimes_2024.csv', index=False)
print("\nTable saved as 'top_20_property_crimes_2024.csv'")

## 7. Summary Statistics

In [ ]:
# Print summary
print("=" * 80)
print("ANALYSIS SUMMARY")
print("=" * 80)

# Violent crimes summary
if len(df_violent_table) > 0:
    avg_growth_violent = df_violent_table['Growth 2019-2024 (%)'].mean()
    print(f"\nViolent Crimes (Top 20):")
    print(f"  - Average growth 2019-2024: {avg_growth_violent:+.1f}%")
    print(f"  - Highest crime rate: {df_violent_table['Violent Crime Violation'].iloc[0]}")
    print(f"    Rate: {df_violent_table['Crime Rate 2024'].iloc[0]:,.1f} per 100,000")

# Property crimes summary
if len(df_property_table) > 0:
    avg_growth_property = df_property_table['Growth 2019-2024 (%)'].mean()
    print(f"\nProperty Crimes (Top 20):")
    print(f"  - Average growth 2019-2024: {avg_growth_property:+.1f}%")
    print(f"  - Highest crime rate: {df_property_table['Property Crime Violation'].iloc[0]}")
    print(f"    Rate: {df_property_table['Crime Rate 2024'].iloc[0]:,.1f} per 100,000")

# Overall trend
if len(total_trend) > 0:
    rate_2000 = total_trend[total_trend['Year'] == 2000]['Total Crime Rate'].values
    rate_2024 = total_trend[total_trend['Year'] == 2024]['Total Crime Rate'].values
    if len(rate_2000) > 0 and len(rate_2024) > 0:
        overall_change = ((rate_2024[0] - rate_2000[0]) / rate_2000[0]) * 100
        print(f"\nOverall Crime Rate Change (2000-2024): {overall_change:+.1f}%")

print("\n" + "=" * 80)
print("Analysis complete. Files saved:")
print("  - crime_rate_trends_2000_2024.png")
print("  - top_20_violent_crimes_2024.csv")
print("  - top_20_property_crimes_2024.csv")
print("=" * 80)